In [122]:
# import necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Cleaning rice price from other countries

In [123]:
df_vietnam = pd.read_excel('../data/Export_prices_Thailand_Vietnam_India_and_Pakistan.xlsx', sheet_name = 'Table 27', skiprows = [0,2,3,489, 490, 491,492, 493, 494,495])
df_vietnam.head()

,Unnamed: 0,Unnamed: 1,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa
0,1997/98:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,August,300,283,271,255,NQ,315,NQ
3,NaN,September,300,280,270,255,NQ,315,NQ
4,NaN,October,290,274,248,233,NQ,308,NQ


In [124]:
df_vietnam = df_vietnam.dropna(how='all')
df_vietnam.columns

Index(['Unnamed: 0', 'Unnamed: 1', '5-percent', '10-percent', '15-percent',
       '25-percent', '100-percent', '5-percent.1', 'Pusa'],
      dtype='object')

In [125]:
df_vietnam.rename(columns={'Unnamed: 0': 'Year', 'Unnamed: 1': 'Month'}, inplace=True)
df_vietnam.head(20)

,Year,Month,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa
0,1997/98:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,August,300,283,271,255,NQ,315,NQ
3,NaN,September,300,280,270,255,NQ,315,NQ
4,NaN,October,290,274,248,233,NQ,308,NQ
5,NaN,November,280,270,250,235,NQ,290,NQ
6,NaN,December,278,268,250,238,NQ,290,NQ
7,NaN,January,273,263,250,238,NQ,285,NQ
8,NaN,February,270,260,250,235,NQ,280,NQ
9,NaN,March,277,272,257,242,NQ,280,NQ
10,NaN,April,280,275,260,245,NQ,268,NQ


In [126]:
df_vietnam = df_vietnam[~df_vietnam['Month'].str.contains("Average", na=False)]
df_vietnam.head(20)

,Year,Month,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa
0,1997/98:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,August,300,283,271,255,NQ,315,NQ
3,NaN,September,300,280,270,255,NQ,315,NQ
4,NaN,October,290,274,248,233,NQ,308,NQ
5,NaN,November,280,270,250,235,NQ,290,NQ
6,NaN,December,278,268,250,238,NQ,290,NQ
7,NaN,January,273,263,250,238,NQ,285,NQ
8,NaN,February,270,260,250,235,NQ,280,NQ
9,NaN,March,277,272,257,242,NQ,280,NQ
10,NaN,April,280,275,260,245,NQ,268,NQ


In [127]:
# Forward fill the "Year" column to propagate each year group
df_vietnam["Year"] = df_vietnam["Year"].ffill()

# Initialize an empty list to store dates
dates = []

# Process each row to create the Date column
for i, row in df_vietnam.iterrows():
    year = row["Year"]
    month = row["Month"]
    
    if pd.isna(month) or pd.isna(year):
        dates.append(None)  # Skip if either Year or Month is NaN
        continue
    
    # Extract start and end years from the "Year" format (e.g., "1997/98")
    start_year = int(year.split("/")[0])
    end_year = start_year + 1  # End year is the next year
    
    # Assign the correct year based on the month
    if month in ["August", "September", "October", "November", "December"]:
        full_date = f"{month[:3]}-{start_year}"  # e.g., "Aug-1997"
    else:
        full_date = f"{month[:3]}-{end_year}"  # e.g., "Jan-1998"
    
    dates.append(full_date)

# Add the Date column to the DataFrame
df_vietnam["Date"] = dates

# Drop the original "Year" and "Month" columns if no longer needed
df_vietnam = df_vietnam.drop(columns=["Year", "Month"])

# Display the modified DataFrame
df_vietnam


,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa,Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,300,283,271,255,NQ,315,NQ,Aug-1997
3,300,280,270,255,NQ,315,NQ,Sep-1997
4,290,274,248,233,NQ,308,NQ,Oct-1997
5,280,270,250,235,NQ,290,NQ,Nov-1997
...,...,...,...,...,...,...,...,...
478,NQ,NQ,NQ,NQ,NQ,510,1550,Oct-2023
479,NQ,NQ,NQ,NQ,NQ,500,1550,Nov-2023
480,NQ,NQ,NQ,NQ,NQ,513,1400,Dec-2023
481,NQ,NQ,NQ,NQ,NQ,538,1400,Jan-2024


In [128]:
df_vietnam = df_vietnam.dropna(subset=['Date'])
df_vietnam

,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa,Date
2,300,283,271,255,NQ,315,NQ,Aug-1997
3,300,280,270,255,NQ,315,NQ,Sep-1997
4,290,274,248,233,NQ,308,NQ,Oct-1997
5,280,270,250,235,NQ,290,NQ,Nov-1997
6,278,268,250,238,NQ,290,NQ,Dec-1997
...,...,...,...,...,...,...,...,...
478,NQ,NQ,NQ,NQ,NQ,510,1550,Oct-2023
479,NQ,NQ,NQ,NQ,NQ,500,1550,Nov-2023
480,NQ,NQ,NQ,NQ,NQ,513,1400,Dec-2023
481,NQ,NQ,NQ,NQ,NQ,538,1400,Jan-2024


In [129]:
df_vietnam.isna().sum()

5-percent      0
10-percent     0
15-percent     0
25-percent     0
100-percent    0
5-percent.1    0
Pusa           0
Date           0
dtype: int64

In [130]:
df_vietnam

,5-percent,10-percent,15-percent,25-percent,100-percent,5-percent.1,Pusa,Date
2,300,283,271,255,NQ,315,NQ,Aug-1997
3,300,280,270,255,NQ,315,NQ,Sep-1997
4,290,274,248,233,NQ,308,NQ,Oct-1997
5,280,270,250,235,NQ,290,NQ,Nov-1997
6,278,268,250,238,NQ,290,NQ,Dec-1997
...,...,...,...,...,...,...,...,...
478,NQ,NQ,NQ,NQ,NQ,510,1550,Oct-2023
479,NQ,NQ,NQ,NQ,NQ,500,1550,Nov-2023
480,NQ,NQ,NQ,NQ,NQ,513,1400,Dec-2023
481,NQ,NQ,NQ,NQ,NQ,538,1400,Jan-2024


In [131]:
# only 16 out of 319 is NQ, let's do K Means to estimate values, lets select only some columns
df_vietnam = df_vietnam[['Date', '5-percent', '10-percent', '15-percent', '25-percent']] 
df_vietnam


,Date,5-percent,10-percent,15-percent,25-percent
2,Aug-1997,300,283,271,255
3,Sep-1997,300,280,270,255
4,Oct-1997,290,274,248,233
5,Nov-1997,280,270,250,235
6,Dec-1997,278,268,250,238
...,...,...,...,...,...
478,Oct-2023,NQ,NQ,NQ,NQ
479,Nov-2023,NQ,NQ,NQ,NQ
480,Dec-2023,NQ,NQ,NQ,NQ
481,Jan-2024,NQ,NQ,NQ,NQ


In [132]:
# Replace any value containing 'NQ' with NaN
df_vietnam = df_vietnam.applymap(lambda x: np.nan if isinstance(x, str) and 'NQ' in x else x)
df_vietnam.isna().sum()

/var/folders/5n/m32d0hsx4959r42x3mssyjj40000gn/T/ipykernel_29648/3552525240.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_vietnam = df_vietnam.applymap(lambda x: np.nan if isinstance(x, str) and 'NQ' in x else x)


Date            0
5-percent     110
10-percent    257
15-percent    251
25-percent     59
dtype: int64

In [133]:
from sklearn.impute import KNNImputer

# Convert columns to float, excluding the 'Date' column
for column in df_vietnam.columns[1:]:  # Exclude 'Date' column
    df_vietnam[column] = df_vietnam[column].astype(float)

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=3)

# Perform KNN imputation (excluding the 'Date' column)
df_vietnam_imputed = df_vietnam.copy()  # Make a copy to keep the original Date column
df_vietnam_imputed.iloc[:, 1:] = imputer.fit_transform(df_vietnam.iloc[:, 1:])

# Now df_vietnam_imputed has imputed values for missing entries

In [134]:
df_vietnam_imputed.rename(columns={'5-percent': 'India_White_5%'}, inplace=True)
df_vietnam_imputed = df_vietnam_imputed[['Date', 'India_White_5%']]
df_vietnam_imputed.head()

,Date,India_White_5%
2,Aug-1997,300.0
3,Sep-1997,300.0
4,Oct-1997,290.0
5,Nov-1997,280.0
6,Dec-1997,278.0


In [135]:
df_vietnam_imputed['Date'] = pd.to_datetime(df_vietnam_imputed['Date'] + '-15', format='%b-%Y-%d')
df_vietnam_imputed

,Date,India_White_5%
2,1997-08-15,300.000000
3,1997-09-15,300.000000
4,1997-10-15,290.000000
5,1997-11-15,280.000000
6,1997-12-15,278.000000
...,...,...
478,2023-10-15,345.633484
479,2023-11-15,345.633484
480,2023-12-15,345.633484
481,2024-01-15,345.633484


In [136]:
df_vietnam_imputed.isna().sum()

Date              0
India_White_5%    0
dtype: int64

In [137]:
df_vietnam_imputed.to_csv('../data/tidyIndiaPrice.csv', index = False)